In [8]:
# phase vocoder example https://audioprograming.wordpress.com/2012/03/02/a-phase-vocoder-in-python/
import sys
from scipy import *
from pylab import *
from scipy.io import wavfile
import pandas as pd

def stretch(sound_array, f):
    N = 256 #window_size
    win = np.hanning(N)
    H = N/4
    # read input and get the timescale factor
    (sr,signalin) = wavfile.read(sound_array)
    try:
        signalin = signalin[:,1]
    except:
        pass
        
    L = len(signalin)
    print(L)
    tscale = float(f)
    # signal blocks for processing and output
    phi = np.zeros(N)
    out = np.zeros(N, dtype=complex)
    sigout = np.zeros(L/tscale+N)
    try:
        signalin = signalin[:,1]
    except:
        pass
    # max input amp, window
    amp = max(signalin)
    
    p = 0
    pp = 0

    while p < L-(N+H):
        # take the spectra of two consecutive windows
        p1 = int(p)
        spec1 =  fft(win*signalin[p1:p1+N])
        spec2 =  fft(win*signalin[p1+H:p1+N+H])
        # take their phase difference and integrate
        phi += (angle(spec2) - angle(spec1))
        # bring the phase back to between pi and -pi
        while phi[0]< -pi: 
            phi += 2*pi
        while phi[0] >= pi: 
            phi -= 2*pi
        out.real, out.imag = cos(phi), sin(phi)
        # inverse FFT and overlap-add
        sigout[pp:pp+N] += win*ifft(abs(spec2)*out)
        pp += H
        p += H*tscale

    #  write file to output, scaling it to original amp
    wavfile.write(str(f)+'@'+sound_array,sr,array(amp*sigout/max(sigout), dtype='int16'))


sounds = pd.read_csv('files.csv',sep=',',encoding='gbk')
for s in sounds.iterrows():
    stretch(s[1].file,s[1].speed)

42453
42453
163920
163920


In [ ]:
def stretch(sound_array, f, window_size, h):
    """ 将音频按系数`f`拉伸 """
 
    phase  = np.zeros(window_size)
    hanning_window = np.hanning(window_size)
    result = np.zeros( len(sound_array) /f + window_size)
 
    for i in np.arange(0, len(sound_array)-(window_size+h), h*f):
 
        # 两个可能互相重叠的子数列
        a1 = sound_array[i: i + window_size]
        a2 = sound_array[i + h: i + window_size + h]
 
        # 按第一个数列重新同步第二个数列
        s1 =  np.fft.fft(hanning_window * a1)
        s2 =  np.fft.fft(hanning_window * a2)
        phase = (phase + np.angle(s2/s1)) % 2*np.pi
        a2_rephased = np.fft.ifft(np.abs(s2)*np.exp(1j*phase))
 
        # 加入到结果中
        i2 = int(i/f)
        result[i2 : i2 + window_size] += hanning_window*a2_rephased
 
    result = ((2**(16-4)) * result/result.max()) # 归一化 (16bit)
 
    return result.astype('int16')